In [1]:
import sys
import os
import numpy as np
cwd = os.getcwd()
#Add the directory of the module to the path.
sys.path.append('/'.join(cwd.split('/')[0:-1]))
from virtualMACS import virtualMACS
import mcstasscript as ms

# Initialize the object with a clear name and a cif file

In [2]:
testobj = virtualMACS('test_experiment',cifName='TiO2.cif',useOld=False)
testobj.sample.formula_weight=79.87
#File I/O operations require sudo access. Update to your password below.
testobj.sudo_password='password'

 Material.formula_weight=(val)
#########################
Old simulations found in /mnt/c/Users/tjh/Documents/GitHub/pyMACS/pyMACS/Demonstration/test_experiment/Kidney_simulations/
 
Successfully combined old simulations into /mnt/c/Users/tjh/Documents/GitHub/pyMACS/pyMACS/Demonstration/test_experiment/Kidney_simulations/test_experiment_total.csv

Data matrix instantiated and ready to use.
#########################


## Set the sample parameters

In this example, we use a box of the same dimensions as in the reference experiment. (4.3mm  x 3.3mm x 1.3 mm) (x,y,z)

In [3]:
testobj.sample.sample_shape='box'
testobj.sample.sample_widx=4.3e-3 
testobj.sample.sample_widy=1.3e-3
testobj.sample.sample_widz=3.3e-3 
#Going to make our sample artifically big to increase count rate. 
testobj.sample.sample_widx=5e-3 
testobj.sample.sample_widy=5e-3
testobj.sample.sample_widz=5e-3 

testobj.sample.sample_tilt=-30
'''
testobj.sample.sample_shape='cylinder'
testobj.sample.sample_length=0.02
testobj.sample.sample_diameter_d=0.01
'''

"\ntestobj.sample.sample_shape='cylinder'\ntestobj.sample.sample_length=0.02\ntestobj.sample.sample_diameter_d=0.01\n"

## Assign Monochromator Parameters
In this example, we use Ei=5meV = Ef

In [4]:
testobj.monochromator.Ei = 5.0
testobj.monochromator.Ef = 5.0

## Assign Kidney Parameters

In [5]:
testobj.kidney.Ef=5.0
testobj.kidney.kidney_angle=-10.0

## Checking sample orientation and projection into lab frame is correct.

In [6]:
#First need to convert CIF to lau style file. 
testobj.sample.cif2lau()
Qmod_110 = testobj.sample.Qmag_HKL(1,1,0)
print('Momentum transfer of (110) = '+'{:.2f}'.format(Qmod_110)+' Ang^-1')

 
Conversion of CIF to crystallographical LAU file successful. 
Momentum transfer of (110) = 1.93 Ang^-1


In [7]:
#Tilt the sample
testobj.sample.crystal_axis_xrot=0.0
testobj.sample.crystal_axis_zrot=0.0
testobj.sample.crystal_axis_zrot=0.0

print('Sample Lattice vectors')
print('')
print('a='+str(testobj.sample.a))
print('alpha='+str(testobj.sample.alpha))
print('b='+str(testobj.sample.b))
print('beta='+str(testobj.sample.beta))
print('c='+str(testobj.sample.c))
print('gamma='+str(testobj.sample.gamma))
print('')
print('Sample orientation U')
print(testobj.sample.orient_u)
testobj.sample.orient_u=[1,1,0]
testobj.sample.orient_v=[0,0,1]
print('Sample orientation v')
print(testobj.sample.orient_v)
print('')
testobj.sample.project_sample_realspace()
print('Real Space projection of lattice vectors [ax,ay,az; bx,by,bz;cx,cy,cz]')
print(testobj.sample.labframe_mat)
print('')
print('Structure factors:')
print('|F(110)|^2 = '+str(round(testobj.sample.fetch_F_HKL(1,1,0)[3],4))+' barn')
print('|F(100)|^2 = '+str(round(testobj.sample.fetch_F_HKL(1,0,0)[3],4))+' barn')
print('|F(1-10)|^2 = '+str(round(testobj.sample.fetch_F_HKL(1,-1,0)[3],4))+' barn')
print('|F(001)|^2 = '+str(round(testobj.sample.fetch_F_HKL(0,0,1)[3],4))+' barn')



Sample Lattice vectors

a=4.6001
alpha=90.0
b=4.6001
beta=90.0
c=2.9288
gamma=90.0

Sample orientation U
[1, 1, 0]
Sample orientation v
[0, 0, 1]

Real Space projection of lattice vectors [ax,ay,az; bx,by,bz;cx,cy,cz]
[[ 3.25276 -3.25276 -0.     ]
 [ 3.25276  3.25276 -0.     ]
 [ 0.       0.       2.9288 ]]

Structure factors:
|F(110)|^2 = 0.1782 barn
|F(100)|^2 = 0.0 barn
|F(1-10)|^2 = 0.1782 barn
|F(001)|^2 = 0.0 barn


## Check that some sample dependent cross sections are calculated correctly

In [8]:
print('sigma_abs ='+str(testobj.sample.rho_abs)+' barn/unit cell')
print('sigma_inc ='+str(testobj.sample.sigma_inc)+' barn/unit cell')

sigma_abs =12.18076 barn/unit cell
sigma_inc =2.8716 barn/unit cell


The scripting methodology requires that the scattering processes and the scattering geometries be defined seperately. Any UNION processes and geometries are allowed, see the McStas documentation for details. The respective traces are generated using McStasScript like so

In [9]:
scattering_def = ms.McStas_instr("scattering_definition",checks=False)
inc_scatter = scattering_def.add_component("inc_scatter","Incoherent_process")
inc_scatter.sigma=testobj.sample.sigma_inc
inc_scatter.unit_cell_volume = testobj.sample.cell_vol
inc_scatter.packing_factor = 1
inc_scatter.set_AT([0,0,0])

#Single crystal process. 
crystal_scatter = scattering_def.add_component("crystal_scatter","Single_crystal_process")
crystal_scatter.delta_d_d=0.005
crystal_scatter.mosaic = 30.0
#Projections of lattice vectors onto lab frame is handled by the previous helper process.
labproj = testobj.sample.labframe_mat
crystal_scatter.ax = labproj[0,0]
crystal_scatter.ay = labproj[0,1]
crystal_scatter.az = labproj[0,2]
crystal_scatter.bx = labproj[1,0]
crystal_scatter.by = labproj[1,1]
crystal_scatter.bz = labproj[1,2]
crystal_scatter.cx = labproj[2,0]
crystal_scatter.cy = labproj[2,1]
crystal_scatter.cz = labproj[2,2]
crystal_scatter.reflections='\"'+"TiO2.lau"+'\"'
crystal_scatter.barns=1
crystal_scatter.packing_factor=1
crystal_scatter.powder=0
crystal_scatter.PG=0
crystal_scatter.interact_fraction=0.8
crystal_scatter.set_AT([0,0,0])
crystal_scatter.set_ROTATED([0,0,0])

scattering = scattering_def.add_component("TiO2","Union_make_material")
scattering.process_string='"crystal_scatter,inc_scatter"'
scattering.my_absorption=testobj.sample.rho_abs
scattering.set_AT([0,0,0])

#Now, this pseudo-instrument will be saved as the scattering definition of the sample. 
testobj.sample.scattering_def = scattering_def

#Make a second object for the geometry. This particular case replicates the validation experiment for this package.
geo_def = ms.McStas_instr("geometry_definition",checks=False)

sample_cube=geo_def.add_component("sample_cube","Union_box")
sample_cube.xwidth=1.0*testobj.sample.sample_widx
sample_cube.yheight=1.0*testobj.sample.sample_widy
sample_cube.zdepth=1.0*testobj.sample.sample_widz
sample_cube.priority=100
sample_cube.material_string='\"TiO2\"'
sample_cube.number_of_activations="number_of_activations_sample" #Do not change. 
sample_cube.set_AT([0,0,0],RELATIVE='crystal_assembly')
sample_cube.set_ROTATED([0,0,0],RELATIVE='crystal_assembly')
'''
sample_cube_mask1 = geo_def.add_component("sample_cube_mask1","Union_box") #It's easier to rotate a mask rather than the sample itself.
sample_cube_mask1.xwidth=testobj.sample.sample_widx
sample_cube_mask1.yheight=testobj.sample.sample_widy
sample_cube_mask1.zdepth=testobj.sample.sample_widz
sample_cube_mask1.priority=0
sample_cube_mask1.material_string='"Mask"'
sample_cube_mask1.number_of_activations="number_of_activations_sample"
sample_cube_mask1.mask_string='"sample_cube"'
sample_cube_mask1.mask_setting='"All"'
sample_cube_mask1.visualize=0
sample_cube_mask1.set_AT([0,0,0],RELATIVE="crystal_assembly")
sample_cube_mask1.set_ROTATED([0,0,0], RELATIVE="crystal_assembly")
'''
sample_plate = geo_def.add_component("sample_plate","Union_cylinder")
sample_plate.radius=0.006
sample_plate.yheight=0.002
sample_plate.priority=40
sample_plate.material_string='"Al"'
plate_distance = testobj.sample.sample_widy+0.002
sample_plate.set_AT([0,plate_distance,0],RELATIVE="target")
sample_plate.set_ROTATED([0,0,0],RELATIVE="target")

sample_plate_rod = geo_def.add_component("sample_plate_rod","Union_cylinder")
sample_plate_rod.radius=0.00125
sample_plate_rod.yheight=0.0633
sample_plate_rod.priority=41
sample_plate_rod.material_string='"Al"'
sample_plate_rod.set_AT([0,plate_distance+0.001+0.031,0], RELATIVE="target")
sample_plate_rod.set_ROTATED([0,0,0],RELATIVE="target")

sample_base = geo_def.add_component("sample_base","Union_cylinder")
sample_base.radius=0.0065
sample_base.yheight=0.013
sample_base.priority=42
sample_base.material_string='\"Al\"'
sample_base.set_AT([0,0.0628,0],RELATIVE="target")
sample_base.set_ROTATED([0,0,0],RELATIVE="target")

sample_base_gap = geo_def.add_component("sample_base_gap","Union_cylinder")
sample_base_gap.radius=0.004
sample_base_gap.yheight=0.009
sample_base_gap.priority=43
sample_base_gap.material_string='"Vacuum"'
sample_base_gap.set_AT([0,0.0668,0], RELATIVE="target")
sample_base_gap.set_ROTATED([0,0,0],RELATIVE="target")

testobj.sample.geometry_def = geo_def


## If the instrument file has not been prepared and compiled, do so now.

In [14]:
testobj.instr_main_file

AttributeError: 'virtualMACS' object has no attribute 'instr_main_file'

In [10]:
useOld=True
if useOld==True:
    testobj.prepare_old_expt_directory()
    #testobj.clean_expt_directory()
else:
    testobj.data.data_matrix=False
    #testobj.clean_expt_directory()
    testobj.prepare_expt_directory()
    testobj.edit_instr_file()
    testobj.compileMonochromator()

    testobj.compileInstr()


testobj.n_mono=1e7
testobj.n_sample=1e6

## To clean the directory and prepare a new virtual experiment, try something like the following.

In [11]:
#The following parameters should produce a signal from the (110) on SPEC18
'''
testobj.monochromator.Ei=9.078
testobj.kidney.Ef=9.078
testobj.A3_angle = 67.37
testobj.kidney.kidney_angle=3.0
testobj.preserve_kidney_scan_files=False

testobj.runMonoScan()
testobj.runKidneyScan()
'''
#The files are saved as csv's for now, add to the data_matrix manually.
#testobj.data.load_data_matrix_from_csv('_tio2_a3scan_cube_dataMatrix.csv')


'\ntestobj.monochromator.Ei=9.078\ntestobj.kidney.Ef=9.078\ntestobj.A3_angle = 67.37\ntestobj.kidney.kidney_angle=3.0\ntestobj.preserve_kidney_scan_files=False\n\ntestobj.runMonoScan()\ntestobj.runKidneyScan()\n'


Limited MSlice-like tools are also available. To use these, first project the measurement into Q-space. Real data may also be loaded to compare to.

In [12]:

testobj.data.data_matrix


,Unnamed: 0.1,Unnamed: 0,A3,A2,A5,A6,H,K,L,Ei,...,SPEC11Err,SPEC12Err,SPEC13Err,SPEC14Err,SPEC15Err,SPEC16Err,SPEC17Err,SPEC18Err,SPEC19Err,SPEC20Err
0,0,0,-67.25,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0
1,0,0,-67.50,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,6447.988017,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0
2,0,0,-67.75,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0
3,0,0,-68.00,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0
4,0,0,-68.25,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,192,192,-69.00,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,320.888531,8.025826,4.363670,0.229677,0.000000e+00,0.000000,0.000000,3.592318,9.341025e-07,0.0
1593,193,193,-69.25,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,341.426454,7.770039,3.856307,12.121509,9.160756e-07,0.000000,0.000000,0.000000,0.000000e+00,0.0
1594,194,194,-69.50,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,315.621893,5.983626,0.828727,133.406083,0.000000e+00,0.894694,0.000000,3.256799,0.000000e+00,0.0
1595,195,195,-69.75,74.156993,37.078496,74.156993,NaN,NaN,NaN,5.0,...,347.943505,12.595845,0.481072,0.000000,3.281168e+00,0.000000,0.000000,0.000000,0.000000e+00,0.0


## Scripting is simple. Simply specify a set of A3 angles and a list of incident energies and the package will handle the rest.

In [13]:
testobj.n_sample=1e5
testobj.n_mono = 1e8
testobj.kidney.Ef=5.0

testobj.kidney_angle_resolution=0.8
testobj.preserve_kidney_scan_files=False
testobj.script_scan(A3_list=np.arange(-70,-20,0.25),Ei_list=[5.0],\
                    num_threads=12,scan_title='_tio2_a3scan_cube')

/dev/shm/memory successfully mounted.
Running these Ei values:[5.]


Total Scans:   0%|          | 0/1 [00:00<?, ?it/s]

Ei=5.0 meV:   0%|          | 0/12 [00:00<?, ?it/s]

Found previous identical kidney simulation. If this is a mistake, 
delete the scan from the data matrix and try again. 
/dev/shm/memory/test_experiment_kidney_angle_m28p5053_Ei_5p0000_Ef_5p0000_A3_angle_m67p0000_sample_diameter_d_0p0200
Found previous identical kidney simulation. If this is a mistake, 
delete the scan from the data matrix and try again. 
/dev/shm/memory/test_experiment_kidney_angle_m28p5053_Ei_5p0000_Ef_5p0000_A3_angle_m66p5000_sample_diameter_d_0p0200
Found previous identical kidney simulation. If this is a mistake, 
delete the scan from the data matrix and try again. 
/dev/shm/memory/test_experiment_kidney_angle_m28p5053_Ei_5p0000_Ef_5p0000_A3_angle_m66p0000_sample_diameter_d_0p0200
Found previous identical kidney simulation. If this is a mistake, 
delete the scan from the data matrix and try again. 
/dev/shm/memory/test_experiment_kidney_angle_m28p5053_Ei_5p0000_Ef_5p0000_A3_angle_m65p7500_sample_diameter_d_0p0200
Found previous identical kidney simulation. If this 

ValueError: No objects to concatenate

In [ ]:
testobj.data.load_data_matrix_from_csv('_tio2_a3scan_cube_dataMatrix.csv')
testobj.data.write_data_to_ng0('tio2_a3scan_cube.ng0')


import matplotlib.pyplot as plt
testobj.data.project_data_QE()
U,V,I = testobj.data.bin_constE_slice(120,120,[-2,2],[-2,2],[-1,1])

plt.figure()
plt.pcolormesh(U,V,I.T,vmin=0,vmax=20)
plt.xlabel('[HH0]')
plt.ylabel('[00L]')
plt.title("TiO2 A3 Scan, Elastic")

## It is also simple to emulate an experimental scan using the same values of A3, kidney angle, Ei, and Ef. Parameters are copied directly from ng0 file.

In [ ]:
testobj.data.data_matrix=False 
#object looks for previous scans in the data matrix, set the data matrix to False to disable this behavior and run 
# all scans regardless of if they have been run before.
sample_ng0 = 'Example_ng0_files/fpx78891.ng0'
testobj.n_sample=1e6
testobj.simulate_ng0(sample_ng0,n_threads=12)



## We can also do this for a directory of ng0 files, this will take a while. 

In [ ]:
ngo_dir = 'Example_ng0_files/'
testobj.simulate_ng0dir(ngo_dir,n_threads=12) #Uncomment this line to run the example directory.

## At any point the files in the kidney scan folder can be converted into MSlice readable ng0 files. The files may be divided into individual Ei values or combined into a single larger one. If they originate from ng0 files, they may also be individual ng0 files corresponding to their origin files

In [ ]:
#Here we combine any scans that exist individually and append them to the data holder class
testobj.data.combine_csv_scans(preserve_old=True,flagstr='_combined_')
testobj.data.load_data_matrix_from_csv(csv_name='_combined_dataMatrix.csv')

In [ ]:
#The data is now written to a MACS style file for comparison in MSlice.
testobj.data.write_data_to_ng0(filename='_cube_TiO2_demonstration_scan.ng0',beta_1=testobj.monochromator.beta_1,\
                               beta_2=testobj.monochromator.beta_2)

In [ ]:
testobj.data.combine_all_csv()
testobj.data.load_data_matrix_from_csv('_total.csv')

In [ ]:
testobj.data.write_data_to_ng0(filename='_a3scan.ng0')